# CNA news crawling: step by step

中央社新聞爬蟲-分解動作

Only 20 pieces of news for each category in a page

Beautiful Soup is a HTML/XML parser with Python
Beautiful Soup是用Python寫的一個HTML/XML的解析器

    find specified tag with the following appraoches:
    尋找特定標籤或有著特定屬性的標籤Tag，有以下兩種方法

    (1)find
    find('article', {'class':"article"})
    findAll('' , {})



    (2)select 
    select_one()
    select()


    findAll():  find all 找到全部
    find():    find one 找到一個就停止

    連續查找:
    find().find()

Reference參考資料
https://www.crummy.com/software/BeautifulSoup/bs3/documentation.html

In [2]:
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent

通常這個類別來自於像是 [fake-useragent](https://pypi.org/project/fake-useragent/) 這樣的套件，用於隨機產生瀏覽器用的 User-Agent 字串。這樣在進行網路爬蟲或 HTTP 請求時，可以模擬不同的瀏覽器，幫助避免被目標網站封鎖或識別為機器人。

In [3]:

user_agent = UserAgent()

In [4]:
user_agent.random

'Mozilla/5.0 (iPhone; CPU iPhone OS 18_1_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.1.1 Mobile/15E148 Safari/604.1'

# Crawl title and links from a certain page

爬取某一類新聞標題與連結(以政治新聞為例)

## Political news

以政治新聞為例

In [5]:
category_url = 'https://edition.cnn.com/politics'
# req = requests.get( category_url )
req = requests.get(category_url, headers={ 'user-agent': user_agent.random }, timeout=5)

In [6]:
req

<Response [200]>

In [7]:
req.text

'  <!DOCTYPE html>\n<html lang="en" data-uri="cms.cnn.com/_pages/clbdmn72h000067qi92erqq0s@published" data-layout-uri="cms.cnn.com/_layouts/layout-no-rail/instances/new-politics-v1@published" >\n  <head>\n<link rel="dns-prefetch" href="//tpc.googlesyndication.com">\n\n<link rel="preconnect" href="//tpc.googlesyndication.com">\n\n<link rel="dns-prefetch" href="//pagead2.googlesyndication.com">\n\n<link rel="preconnect" href="//pagead2.googlesyndication.com">\n\n<link rel="dns-prefetch" href="//www.googletagservices.com">\n\n<link rel="preconnect" href="//www.googletagservices.com">\n\n<link rel="dns-prefetch" href="//www.google.com">\n\n<link rel="preconnect" href="//www.google.com">\n\n<link rel="dns-prefetch" href="//c.amazon-adsystem.com">\n\n<link rel="preconnect" href="//c.amazon-adsystem.com">\n\n<link rel="dns-prefetch" href="//ib.adnxs.com">\n\n<link rel="preconnect" href="//ib.adnxs.com">\n\n<link rel="dns-prefetch" href="//cdn.adsafeprotected.com">\n\n<link rel="preconnect" hr

In [8]:
page = BeautifulSoup(req.text,'lxml')

In [9]:
print(page)

<!DOCTYPE html>
<html data-layout-uri="cms.cnn.com/_layouts/layout-no-rail/instances/new-politics-v1@published" data-uri="cms.cnn.com/_pages/clbdmn72h000067qi92erqq0s@published" lang="en">
<head>
<link href="//tpc.googlesyndication.com" rel="dns-prefetch"/>
<link href="//tpc.googlesyndication.com" rel="preconnect"/>
<link href="//pagead2.googlesyndication.com" rel="dns-prefetch"/>
<link href="//pagead2.googlesyndication.com" rel="preconnect"/>
<link href="//www.googletagservices.com" rel="dns-prefetch"/>
<link href="//www.googletagservices.com" rel="preconnect"/>
<link href="//www.google.com" rel="dns-prefetch"/>
<link href="//www.google.com" rel="preconnect"/>
<link href="//c.amazon-adsystem.com" rel="dns-prefetch"/>
<link href="//c.amazon-adsystem.com" rel="preconnect"/>
<link href="//ib.adnxs.com" rel="dns-prefetch"/>
<link href="//ib.adnxs.com" rel="preconnect"/>
<link href="//cdn.adsafeprotected.com" rel="dns-prefetch"/>
<link href="//cdn.adsafeprotected.com" rel="preconnect"/>
<l

## Where are the news links?

觀察一下網頁內容，你要爬取的連結在哪一個標籤裡面?

<div class="container__field-links container_lead-plus-headlines__field-links" data-editable="cards">

In [9]:
#<div class="container__field-links container_lead-plus-headlines__field-links" data-editable="cards"> 

<div data-uri="cms.cnn.com/_components/card/instances/clbdmol44002m3d6euos0g5zg_fill_2@published" data-created-updated-by="true" class="card container__item container__item--type-media-image container__item--type-section container_lead-plus-headlines__item container_lead-plus-headlines__item--type-section  container_lead-plus-headlines__selected" data-component-name="card" data-open-link="/2025/03/11/politics/department-of-education-cuts/index.html" data-page="cms.cnn.com/_pages/cm84uzsnr002r26qm2h814uwy@published" data-unselectable="true" data-word-count="918">
    
            <a href="/2025/03/11/politics/department-of-education-cuts/index.html" class="container__link container__link--type-article container_lead-plus-headlines__link" data-link-type="article" data-zjs="click" data-zjs-cms_id="cms.cnn.com/_pages/clbdmn72h000067qi92erqq0s@published" data-zjs-canonical_url="https://www.cnn.com/politics" data-zjs-zone_id="cms.cnn.com/_components/zone/instances/clbdmn749000z67qi0k7iz4es@published" data-zjs-zone_name="undefined" data-zjs-zone_type="zone_layout--wide-left-balanced-2" data-zjs-zone_position_number="1" data-zjs-zone_total_number="5" data-zjs-container_id="cms.cnn.com/_components/container/instances/clbdmol44002m3d6euos0g5zg@published" data-zjs-container_name="LATEST HEADLINES" data-zjs-container_type="container_lead-plus-headlines" data-zjs-container_position_number="1" data-zjs-container_total_number="3" data-zjs-card_id="cms.cnn.com/_components/card/instances/clbdmol44002m3d6euos0g5zg_fill_2@published" data-zjs-card_name="Education Department cutting nearly half of workforce" data-zjs-card_type="card" data-zjs-card_position_number="2" data-zjs-card_total_number="8">
                <div class="container__text container_lead-plus-headlines__text">
            <div class="container__headline container_lead-plus-headlines__headline">
                        <!-- This needs to be all one one line or it will cause unwanted spacing due to handlebar output -->
<span class="container__headline-text" data-editable="headline">Education Department cutting nearly half of workforce</span>
            </div>
        </div>
    </a>
</div>

In [14]:
# 從共同的父元素 container 取得兩個部分
a_tag = container.find("a")
span_tag = container.find("span", class_="container__headline-text")

if a_tag and span_tag:
    href = a_tag.get("href")
    new_tag = f'<a href="{href}">{str(span_tag)}</a>'
    print(new_tag)
else:
    print("找不到 a 或 span 標籤")

<a href="/2025/03/12/politics/johnson-loudermilk-trump-january-6-investigation-cheney/index.html"><span class="container__headline-text" data-editable="headline">Speaker Mike Johnson and Republicans are at odds over January 6 investigation</span></a>


In [19]:
news_item = []

for a_tag in container.find_all("a"):
    span_tag = a_tag.find("span", class_="container__headline-text")
    if span_tag:
        href = a_tag.get("href")
        new_tag = f'<a href="{href}">{str(span_tag)}</a>'
        news_item.append(new_tag)

news_item

['<a href="/2025/03/12/politics/johnson-loudermilk-trump-january-6-investigation-cheney/index.html"><span class="container__headline-text" data-editable="headline">Speaker Mike Johnson and Republicans are at odds over January 6 investigation</span></a>',
 '<a href="/2025/03/11/politics/federal-judge-trump-administration-testify-probationary-firings/index.html"><span class="container__headline-text" data-editable="headline">Justice Department says key Trump admin official won’t testify about probationary firings</span></a>',
 '<a href="/2025/03/11/politics/department-of-education-cuts/index.html"><span class="container__headline-text" data-editable="headline">Education Department cutting nearly half of workforce</span></a>',
 '<a href="/2025/03/11/politics/opm-spokesperson-fashion-influencer-videos-invs/index.html"><span class="container__headline-text" data-editable="headline">Trump official tasked with defending DOGE cuts posted fashion influencer videos from her office</span></a>',
 

In [ ]:
# 假設 container 是已經用 BeautifulSoup 建立的元素
news_item = []

for a_tag in container.find_all("a"):
    span_tag = a_tag.find("span", class_="container__headline-text")
    if span_tag:
        href = a_tag.get("href")
        # 建立一個新的 a 標籤（利用 container.new_tag），只設定 href 屬性
        new_a = container.new_tag("a", href=href)
        # 為避免從原本結構移除 span，建立一個新的 BeautifulSoup 物件來複製 span
        span_clone = BeautifulSoup(str(span_tag), "html.parser").find("span")
        new_a.append(span_clone)
        news_item.append(new_a)

# 此時 news_item 裡的每一個元素都是 BeautifulSoup 的 Tag 物件
news_item

[<a href="/2025/03/12/politics/johnson-loudermilk-trump-january-6-investigation-cheney/index.html"><span class="container__headline-text" data-editable="headline">Speaker Mike Johnson and Republicans are at odds over January 6 investigation</span></a>,
 <a href="/2025/03/11/politics/federal-judge-trump-administration-testify-probationary-firings/index.html"><span class="container__headline-text" data-editable="headline">Justice Department says key Trump admin official won’t testify about probationary firings</span></a>,
 <a href="/2025/03/11/politics/department-of-education-cuts/index.html"><span class="container__headline-text" data-editable="headline">Education Department cutting nearly half of workforce</span></a>,
 <a href="/2025/03/11/politics/opm-spokesperson-fashion-influencer-videos-invs/index.html"><span class="container__headline-text" data-editable="headline">Trump official tasked with defending DOGE cuts posted fashion influencer videos from her office</span></a>,
 <a href=

In [ ]:
#page.find('ul',{'id':"jsMainList"})

## news link is in the "li" tag

In [ ]:
items = page.find('ul',{'id':"jsMainList"}).findAll('li')
items

### 兩岸新聞改版成 jsMainList_Style2

re.compile("jsMainList\w*") ==> jsMainList, jsMainList_Style2 都會被包含在內

In [ ]:
# items = page.find('ul',{'id':"myMainList"}).findAll('li')  # It will fail
# items = page.find('ul', {'id': re.compile("jsMainList\w*")}).findAll('li') # 

## find all links

In [13]:
items = page.find('ul',{'id':"jsMainList"}).findAll('li')

In [ ]:
items

## Get title

In [32]:
item = news_item[0]

In [33]:
item

<a href="/2025/03/12/politics/johnson-loudermilk-trump-january-6-investigation-cheney/index.html"><span class="container__headline-text" data-editable="headline">Speaker Mike Johnson and Republicans are at odds over January 6 investigation</span></a>

In [36]:
item.find('span').text

'Speaker Mike Johnson and Republicans are at odds over January 6 investigation'

## Get link

In [43]:
item.get('href')


'/2025/03/12/politics/johnson-loudermilk-trump-january-6-investigation-cheney/index.html'

## Get  news time and convert to the format of "2022-04-20"

In [44]:
from datetime import datetime

In [45]:
item.find('div', {'class':"date"}).text

AttributeError: 'NoneType' object has no attribute 'text'

In [21]:
news_time = item.find('div', {'class':"date"}).text
dtime = datetime.strptime(news_time, '%Y/%m/%d %H:%M')
news_date = dtime.strftime("%Y-%m-%d")

In [22]:
news_date

'2025-02-27'

## Get photo_link
    <img src=...
    <img data-scr=...

    <img alt="防覆艇意外  軍方高雄桃子園海灘建置監控系統" src="https://imgcdn.cna.com.tw/www/WebPhotos/400/20210314/1200x903_71068052983.jpg"/>


    <img data-src="https://imgcdn.cna.com.tw/www/WebPhotos/400/20210314/1155x768_20210314000038.jpg" class=" lazyloaded" alt="游錫堃：落實開放國會行動方案  方便人民參與" src="https://imgcdn.cna.com.tw/www/WebPhotos/400/20210314/1155x768_20210314000038.jpg">

In [55]:
item = page.find('div',{'class':"image__container"}).findAll('picture')
item

[<picture class="image__picture"><source height="438" media="(min-width: 1280px)" srcset="https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&amp;q=h_438,w_780,c_fill" width="780"><source height="438" media="(min-width: 960px)" srcset="https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&amp;q=h_438,w_780,c_fill" width="780"><source height="270" media="(min-width: 480px)" srcset="https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&amp;q=h_270,w_480,c_fill" width="480"><source height="270" media="(max-width: 479px)" srcset="https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&amp;q=h_270,w_480,c_fill" width="480"><img alt="In this June 2022 photo, Liz Cheney is seen in the Cannon House Office Building in Washington, DC." class="image__dam-img image__dam-img--loading" height="2000" onerror="imageLoadError(this)" onload="this.classList.remove('image__dam-img--loading')

In [67]:
source_tag = item[0].find('source',{'media':'(min-width: 1280px)'})
source_tag

<source height="438" media="(min-width: 1280px)" srcset="https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&amp;q=h_438,w_780,c_fill" width="780"><source height="438" media="(min-width: 960px)" srcset="https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&amp;q=h_438,w_780,c_fill" width="780"><source height="270" media="(min-width: 480px)" srcset="https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&amp;q=h_270,w_480,c_fill" width="480"><source height="270" media="(max-width: 479px)" srcset="https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&amp;q=h_270,w_480,c_fill" width="480"><img alt="In this June 2022 photo, Liz Cheney is seen in the Cannon House Office Building in Washington, DC." class="image__dam-img image__dam-img--loading" height="2000" onerror="imageLoadError(this)" onload="this.classList.remove('image__dam-img--loading')" src="https://media.cnn.com/api/

In [ ]:
# src
item[0].find('source').get('')

'https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&q=h_438,w_780,c_fill'

In [77]:
# check if element has_attr('data-src')
item[0].find('source').has_attr('srcset')

True

In [78]:
item[0].find('source').has_attr('src')

False

In [81]:
# Use if else to deal with it
if item[0].find('source').has_attr('srcset'):
    photo_link = item[0].find('source').get('srcset')
else:
    photo_link = item[0].find('source').get('src')

In [82]:
photo_link

'https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1405666105.jpg?c=16x9&q=h_438,w_780,c_fill'

## Error occurs if the news has no photo link

NoneType

In [35]:
item = items[3]

In [37]:
item.find('img')

In [38]:
type(item.find('img'))

NoneType

In [39]:
# An error arises!
item.find('img').get('src')

AttributeError: 'NoneType' object has no attribute 'get'

In [40]:
try:
    photo_link=item.find('img').get('src')
except:
    print('No photo link!')
    photo_link=''

No photo link!


In [41]:
try:
    # Use if else to deal with it
    if item.find('img').has_attr('data-src'):
        photo_link = item.find('img').get('data-src')
    else:
        photo_link = item.find('img').get('src')
except:
    print('No photo link!')
    photo_link=''

No photo link!


In [42]:
photo_link

''

# Fail to  get news content Why?

In [43]:
article_url = items[0].find('a').get('href')

In [44]:
article_url 

'/news/aipl/202502270054.aspx'

In [45]:
# 最新的新聞列表之新聞連結:開頭缺少"https://www.cna.com.tw"
if article_url.startswith("/news"):
    article_url ="https://www.cna.com.tw"+article_url

In [46]:
article_url

'https://www.cna.com.tw/news/aipl/202502270054.aspx'

In [47]:
# req = requests.get( article_url )
req = requests.get(article_url, headers={ 'user-agent': user_agent.random }, timeout=5)

page = BeautifulSoup(req.text,'lxml')

In [48]:
page

<!DOCTYPE html>
<html lang="zh-Hant-TW"><head><title>
	罷免19藍委與高虹安一階審核通過 另9案待補件 | 政治 | 中央社 CNA
</title><meta content="全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。" name="description"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<link href="https://www.cna.com.tw/news/aipl/202502270054.aspx" rel="canonical"/>
<meta content="2025-02-27T11:18:00+08:00" property="article:modified_time"/>
<meta content="2025-02-27T10:57:00+08:00" property="article:published_time"/>
<link href="https://imgcdn.cna.com.tw/www/webphotos/WebCover/800/20250227/1067x800_539032842902.jpg" rel="image_src" type="image/jpeg"/>
<meta content="中央通訊社" property="author"/>
<link href="https://plus.google.com/102054703728087773572" rel="author"/>
<link href="https://imgcdn.cna.com.tw/www/websi

# Request page using fake_useragent

In [51]:
# headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}

In [49]:
from fake_useragent import UserAgent
user_agent = UserAgent()

In [50]:
user_agent.random

'Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1'

In [51]:
req = requests.get(article_url, headers={ 'user-agent': user_agent.random }, timeout=5)

In [52]:

page = BeautifulSoup(req.text,'lxml')

In [53]:
page

<!DOCTYPE html>
<html lang="zh-Hant-TW"><head><title>
	罷免19藍委與高虹安一階審核通過 另9案待補件 | 政治 | 中央社 CNA
</title><meta content="全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。" name="description"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<link href="https://www.cna.com.tw/news/aipl/202502270054.aspx" rel="canonical"/>
<meta content="2025-02-27T11:18:00+08:00" property="article:modified_time"/>
<meta content="2025-02-27T10:57:00+08:00" property="article:published_time"/>
<link href="https://imgcdn.cna.com.tw/www/webphotos/WebCover/800/20250227/1067x800_539032842902.jpg" rel="image_src" type="image/jpeg"/>
<meta content="中央通訊社" property="author"/>
<link href="https://plus.google.com/102054703728087773572" rel="author"/>
<link href="https://imgcdn.cna.com.tw/www/websi

In [54]:
content = page.find('div',{'class':"paragraph"})
content

<div class="paragraph"><p>（中央社記者高華謙台北27日電）全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。</p><div class="dictionary"><span>中選會截至今天共收到64案</span><span class="news-dic-inline" data-id="001959"><span>罷免案</span><span class="dic-icon"></span></span><div class="dictionary-box" data-box="001959"><p class="dictionary-title">2025罷免案程序</p><div class="news-dic-text"><p>第一階段：提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊。</p>
<p>第二階段：連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複。</p>
<p>若經查對無誤成案，被罷免人於10日內提出答辯書。提出答辯書期間屆滿後5日內，公告罷免投票日期。</p>
<p>第三階段：罷免案宣告成立後20日起至60日內投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算通過。</p></div><div class="news-dic-all seeArticlemore">看更多</div><div class="news-dic-all closeBox">關閉</div></div><span>送件，其中54案為立委，1案為市縣長罷免案，9案為市縣議員罷免案。</span></div><p>民間團體2月3日赴中選會遞交罷免第一階段提議書，被提議罷免者包括國民黨立委王鴻薇、李彥秀、葉元之、牛煦庭、涂權吉、魯明哲、萬美玲、呂玉玲、邱若華、鄭正鈐、游顥、馬文君、顏寬恒、廖偉翔、黃健豪、羅廷瑋、丁學忠、黃建賓、傅崐萁，以及遭停職的新竹市長高虹安。</p><p>依公職人員選舉罷免法規定，2月3日遞交的罷免案，中選會應於2月28日前審定完畢。</p><p>據了解，中選會昨天

In [55]:
content.text

'（中央社記者高華謙台北27日電）全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。中選會截至今天共收到64案罷免案2025罷免案程序第一階段：提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊。\n第二階段：連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複。\n若經查對無誤成案，被罷免人於10日內提出答辯書。提出答辯書期間屆滿後5日內，公告罷免投票日期。\n第三階段：罷免案宣告成立後20日起至60日內投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算通過。看更多關閉送件，其中54案為立委，1案為市縣長罷免案，9案為市縣議員罷免案。民間團體2月3日赴中選會遞交罷免第一階段提議書，被提議罷免者包括國民黨立委王鴻薇、李彥秀、葉元之、牛煦庭、涂權吉、魯明哲、萬美玲、呂玉玲、邱若華、鄭正鈐、游顥、馬文君、顏寬恒、廖偉翔、黃健豪、羅廷瑋、丁學忠、黃建賓、傅崐萁，以及遭停職的新竹市長高虹安。依公職人員選舉罷免法規定，2月3日遞交的罷免案，中選會應於2月28日前審定完畢。據了解，中選會昨天舉行委員會議，2月3日遞交的20案罷免提議書已審議通過，後續送的另9名立委罷免案皆需補件；其餘罷免案持續由地方選委會查對中。透過 Google News追蹤中央社根據中選會網站，符合門檻規定者，在審議通過後函請提議領銜人於文到次日起10日內至中選會領取連署人名冊格式。如不符門檻規定，則會請其補提，補提以1次為限。根據中選會網站，罷免可分為提議、連署與投票3大階段。第一階段提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊是否正確；若進入第2階段連署，連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複；若進入第3階段投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算罷免通過。（編輯：蘇志宗）1140227#公職人員選舉罷免法#中選會#高虹安#國民黨延伸閱讀大罷免潮 卓榮泰：最終仍盼和諧政治環境內政部提修法 罷免領銜人、被罷免者得收政治獻金內政部：選

## Remove unnecessary elements

Remove "透過 Google News追蹤中央社"


<div class="gmailNews"><div class="gmailNewsWrapper"><img width="78" height="64" src="https://imgcdn.cna.com.tw/www/website/img/google-news.png" alt="google news"><span>透過 Google News</span><a href="https://news.google.com/publications/CAAqBwgKMKD2uwswrZHTAw?hl=zh-TW&amp;gl=TW&amp;ceid=TW%3Azh-Hant" target="_blank">追蹤中央社</a></div></div>

In [60]:
content.find('div', {'class':"gmailNews"})

<div class="gmailNews"><div class="gmailNewsWrapper"><img alt="google news" height="64" src="https://imgcdn.cna.com.tw/www/website/img/google-news.png" width="78"/><span>透過 Google News</span><a href="https://news.google.com/publications/CAAqBwgKMKD2uwswrZHTAw?hl=zh-TW&amp;gl=TW&amp;ceid=TW%3Azh-Hant" target="_blank">追蹤中央社</a></div></div>

In [61]:
len(content.findAll('div', {'class':"gmailNews"}))

1

In [62]:
content.findAll('div', {'class':"gmailNews"})

[<div class="gmailNews"><div class="gmailNewsWrapper"><img alt="google news" height="64" src="https://imgcdn.cna.com.tw/www/website/img/google-news.png" width="78"/><span>透過 Google News</span><a href="https://news.google.com/publications/CAAqBwgKMKD2uwswrZHTAw?hl=zh-TW&amp;gl=TW&amp;ceid=TW%3Azh-Hant" target="_blank">追蹤中央社</a></div></div>]

In [63]:
content.find('div', {'class':"gmailNews"})
content


<div class="paragraph"><p>（中央社記者高華謙台北27日電）全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。</p><div class="dictionary"><span>中選會截至今天共收到64案</span><span class="news-dic-inline" data-id="001959"><span>罷免案</span><span class="dic-icon"></span></span><div class="dictionary-box" data-box="001959"><p class="dictionary-title">2025罷免案程序</p><div class="news-dic-text"><p>第一階段：提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊。</p>
<p>第二階段：連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複。</p>
<p>若經查對無誤成案，被罷免人於10日內提出答辯書。提出答辯書期間屆滿後5日內，公告罷免投票日期。</p>
<p>第三階段：罷免案宣告成立後20日起至60日內投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算通過。</p></div><div class="news-dic-all seeArticlemore">看更多</div><div class="news-dic-all closeBox">關閉</div></div><span>送件，其中54案為立委，1案為市縣長罷免案，9案為市縣議員罷免案。</span></div><p>民間團體2月3日赴中選會遞交罷免第一階段提議書，被提議罷免者包括國民黨立委王鴻薇、李彥秀、葉元之、牛煦庭、涂權吉、魯明哲、萬美玲、呂玉玲、邱若華、鄭正鈐、游顥、馬文君、顏寬恒、廖偉翔、黃健豪、羅廷瑋、丁學忠、黃建賓、傅崐萁，以及遭停職的新竹市長高虹安。</p><p>依公職人員選舉罷免法規定，2月3日遞交的罷免案，中選會應於2月28日前審定完畢。</p><p>據了解，中選會昨天

In [64]:
content.text

'（中央社記者高華謙台北27日電）全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。中選會截至今天共收到64案罷免案2025罷免案程序第一階段：提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊。\n第二階段：連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複。\n若經查對無誤成案，被罷免人於10日內提出答辯書。提出答辯書期間屆滿後5日內，公告罷免投票日期。\n第三階段：罷免案宣告成立後20日起至60日內投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算通過。看更多關閉送件，其中54案為立委，1案為市縣長罷免案，9案為市縣議員罷免案。民間團體2月3日赴中選會遞交罷免第一階段提議書，被提議罷免者包括國民黨立委王鴻薇、李彥秀、葉元之、牛煦庭、涂權吉、魯明哲、萬美玲、呂玉玲、邱若華、鄭正鈐、游顥、馬文君、顏寬恒、廖偉翔、黃健豪、羅廷瑋、丁學忠、黃建賓、傅崐萁，以及遭停職的新竹市長高虹安。依公職人員選舉罷免法規定，2月3日遞交的罷免案，中選會應於2月28日前審定完畢。據了解，中選會昨天舉行委員會議，2月3日遞交的20案罷免提議書已審議通過，後續送的另9名立委罷免案皆需補件；其餘罷免案持續由地方選委會查對中。透過 Google News追蹤中央社根據中選會網站，符合門檻規定者，在審議通過後函請提議領銜人於文到次日起10日內至中選會領取連署人名冊格式。如不符門檻規定，則會請其補提，補提以1次為限。根據中選會網站，罷免可分為提議、連署與投票3大階段。第一階段提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊是否正確；若進入第2階段連署，連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複；若進入第3階段投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算罷免通過。（編輯：蘇志宗）1140227#公職人員選舉罷免法#中選會#高虹安#國民黨延伸閱讀大罷免潮 卓榮泰：最終仍盼和諧政治環境內政部提修法 罷免領銜人、被罷免者得收政治獻金內政部：選

In [65]:
#req = requests.get( article_url )
req = requests.get(article_url, headers={ 'user-agent': user_agent.random }, timeout=5)
page = BeautifulSoup(req.text,'lxml')

content = page.find('div',{'class':"paragraph"})

In [66]:
content

<div class="paragraph"><p>（中央社記者高華謙台北27日電）全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。</p><div class="dictionary"><span>中選會截至今天共收到64案</span><span class="news-dic-inline" data-id="001959"><span>罷免案</span><span class="dic-icon"></span></span><div class="dictionary-box" data-box="001959"><p class="dictionary-title">2025罷免案程序</p><div class="news-dic-text"><p>第一階段：提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊。</p>
<p>第二階段：連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複。</p>
<p>若經查對無誤成案，被罷免人於10日內提出答辯書。提出答辯書期間屆滿後5日內，公告罷免投票日期。</p>
<p>第三階段：罷免案宣告成立後20日起至60日內投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算通過。</p></div><div class="news-dic-all seeArticlemore">看更多</div><div class="news-dic-all closeBox">關閉</div></div><span>送件，其中54案為立委，1案為市縣長罷免案，9案為市縣議員罷免案。</span></div><p>民間團體2月3日赴中選會遞交罷免第一階段提議書，被提議罷免者包括國民黨立委王鴻薇、李彥秀、葉元之、牛煦庭、涂權吉、魯明哲、萬美玲、呂玉玲、邱若華、鄭正鈐、游顥、馬文君、顏寬恒、廖偉翔、黃健豪、羅廷瑋、丁學忠、黃建賓、傅崐萁，以及遭停職的新竹市長高虹安。</p><p>依公職人員選舉罷免法規定，2月3日遞交的罷免案，中選會應於2月28日前審定完畢。</p><p>據了解，中選會昨天

In [67]:
# Remove unnecessary content

# Remove "透過 Google News追蹤中央社"
content = page.find('div',{'class':"paragraph"})

# 額外插入的訂閱連結-->移除之
try:
    content.find('div', {'class':"gmailNews"}).decompose()
except:
    pass
# 額外插入的訂閱連結-->移除之
try:
    content.find('div', {'class':"subscriInnTitle"}).decompose()
except:
    pass

# 額外插入的訂閱連結-->移除之
try:
    content.find('div', {'class':"SubscriptionInner mySubscriptionInner"}).decompose()
except:
    pass

# 額外插入的訂閱連結-->移除之
try:
    content.find('div', {'class':"subscriInnTitle"}).decompose()
except:
    pass
# 額外插入的延伸閱讀區塊-->移除之
try:
    content.find('div', {'class':"paragraph moreArticle"}).decompose()
except:
    pass

# 額外插入的圖片與文字說明區塊-->移除之
try:
    for media in content.findAll('div', {'class':"media"}):
        media.decompose()
except:
    pass

In [68]:
content

<div class="paragraph"><p>（中央社記者高華謙台北27日電）全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。</p><div class="dictionary"><span>中選會截至今天共收到64案</span><span class="news-dic-inline" data-id="001959"><span>罷免案</span><span class="dic-icon"></span></span><div class="dictionary-box" data-box="001959"><p class="dictionary-title">2025罷免案程序</p><div class="news-dic-text"><p>第一階段：提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊。</p>
<p>第二階段：連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複。</p>
<p>若經查對無誤成案，被罷免人於10日內提出答辯書。提出答辯書期間屆滿後5日內，公告罷免投票日期。</p>
<p>第三階段：罷免案宣告成立後20日起至60日內投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算通過。</p></div><div class="news-dic-all seeArticlemore">看更多</div><div class="news-dic-all closeBox">關閉</div></div><span>送件，其中54案為立委，1案為市縣長罷免案，9案為市縣議員罷免案。</span></div><p>民間團體2月3日赴中選會遞交罷免第一階段提議書，被提議罷免者包括國民黨立委王鴻薇、李彥秀、葉元之、牛煦庭、涂權吉、魯明哲、萬美玲、呂玉玲、邱若華、鄭正鈐、游顥、馬文君、顏寬恒、廖偉翔、黃健豪、羅廷瑋、丁學忠、黃建賓、傅崐萁，以及遭停職的新竹市長高虹安。</p><p>依公職人員選舉罷免法規定，2月3日遞交的罷免案，中選會應於2月28日前審定完畢。</p><p>據了解，中選會昨天

In [69]:
cont = content.text
cont 

'（中央社記者高華謙台北27日電）全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。中選會截至今天共收到64案罷免案2025罷免案程序第一階段：提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊。\n第二階段：連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複。\n若經查對無誤成案，被罷免人於10日內提出答辯書。提出答辯書期間屆滿後5日內，公告罷免投票日期。\n第三階段：罷免案宣告成立後20日起至60日內投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算通過。看更多關閉送件，其中54案為立委，1案為市縣長罷免案，9案為市縣議員罷免案。民間團體2月3日赴中選會遞交罷免第一階段提議書，被提議罷免者包括國民黨立委王鴻薇、李彥秀、葉元之、牛煦庭、涂權吉、魯明哲、萬美玲、呂玉玲、邱若華、鄭正鈐、游顥、馬文君、顏寬恒、廖偉翔、黃健豪、羅廷瑋、丁學忠、黃建賓、傅崐萁，以及遭停職的新竹市長高虹安。依公職人員選舉罷免法規定，2月3日遞交的罷免案，中選會應於2月28日前審定完畢。據了解，中選會昨天舉行委員會議，2月3日遞交的20案罷免提議書已審議通過，後續送的另9名立委罷免案皆需補件；其餘罷免案持續由地方選委會查對中。根據中選會網站，符合門檻規定者，在審議通過後函請提議領銜人於文到次日起10日內至中選會領取連署人名冊格式。如不符門檻規定，則會請其補提，補提以1次為限。根據中選會網站，罷免可分為提議、連署與投票3大階段。第一階段提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊是否正確；若進入第2階段連署，連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複；若進入第3階段投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算罷免通過。（編輯：蘇志宗）1140227#公職人員選舉罷免法#中選會#高虹安#國民黨高虹安藍質疑新竹罷免護航林智堅 綠：不要政治揣測柯文哲父親辭世 高虹安到場不發一語'

## Remove unecessary words
There are some words we don't like to analysis, They should be removed.

In [70]:
cont = re.sub('（中央社.*?）','', cont)
cont = re.sub('（編輯：.*','', cont) 
cont = re.sub('（譯者：.*','', cont) 

In [71]:
cont

'全國罷免團體陸續送交第一階段提議書。據了解，中選會昨天舉行委員會議，其中2月3日遞交、針對19個國民黨立委與遭停職的新竹市長高虹安罷免案，第一階段提議書皆審議通過，後續送的另外9名立委罷免案皆需補件。中選會截至今天共收到64案罷免案2025罷免案程序第一階段：提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊。\n第二階段：連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複。\n若經查對無誤成案，被罷免人於10日內提出答辯書。提出答辯書期間屆滿後5日內，公告罷免投票日期。\n第三階段：罷免案宣告成立後20日起至60日內投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算通過。看更多關閉送件，其中54案為立委，1案為市縣長罷免案，9案為市縣議員罷免案。民間團體2月3日赴中選會遞交罷免第一階段提議書，被提議罷免者包括國民黨立委王鴻薇、李彥秀、葉元之、牛煦庭、涂權吉、魯明哲、萬美玲、呂玉玲、邱若華、鄭正鈐、游顥、馬文君、顏寬恒、廖偉翔、黃健豪、羅廷瑋、丁學忠、黃建賓、傅崐萁，以及遭停職的新竹市長高虹安。依公職人員選舉罷免法規定，2月3日遞交的罷免案，中選會應於2月28日前審定完畢。據了解，中選會昨天舉行委員會議，2月3日遞交的20案罷免提議書已審議通過，後續送的另9名立委罷免案皆需補件；其餘罷免案持續由地方選委會查對中。根據中選會網站，符合門檻規定者，在審議通過後函請提議領銜人於文到次日起10日內至中選會領取連署人名冊格式。如不符門檻規定，則會請其補提，補提以1次為限。根據中選會網站，罷免可分為提議、連署與投票3大階段。第一階段提議人數需達原選舉區選舉人總數1%以上，中選會收件後，會在25天內查對提議人名冊是否正確；若進入第2階段連署，連署人數應為原選舉區選舉人總數10%以上，徵求連署書期限為60天，且提議人不得與連署人重複；若進入第3階段投票，有效同意票數須多於不同意票數，且同意票數達原選舉區選舉人總數1/4以上，才算罷免通過。'

# Let's crawl all of them

針對每一個連結項目抓出標題、連結、日期等資訊

In [72]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time

In [73]:

category_url = 'https://www.cna.com.tw/list/aipl.aspx'

# req = requests.get(category_url)
req = requests.get(category_url, headers={ 'user-agent': user_agent.random }, timeout=5)

page = BeautifulSoup(req.text, 'lxml')

In [74]:
req.text

'<!DOCTYPE html><html lang="zh-Hant-TW"><head><title>\r\n\t政治 | 中央社 CNA\r\n</title><meta name="description" content="想掌握國民黨、民進黨等政黨動態、全國與地方選舉布局、公投議題正反交鋒、總統府與行政院會最新消息、立法院議案最新進度、國外代表團訪台與轉型正義議題、CPTPP與RCEP區域整合資訊，鎖定中央社政治新聞準沒錯。" />\r\n<meta charset="utf-8" />\r\n<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0" />\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n<meta http-equiv="X-UA-Compatible" content="IE=11" />\r\n<link href="https://www.cna.com.tw/list/aipl.aspx" rel="canonical" />\r\n<link href="https://imgcdn.cna.com.tw/www/images/pic_fb.jpg?v=20220218" rel="image_src" type="image/jpeg" />\r\n<meta property="author" content="中央通訊社" />\r\n<link href="https://plus.google.com/102054703728087773572" rel="author" />\r\n<link href="https://imgcdn.cna.com.tw/www/website/img/fav-icon.png?v=20250217" type="image/png" rel="icon" />\r\n<meta property="og:url" content="https://www.cna.com.tw/list/aipl.aspx" />\r\n<meta property="og:title" content

In [75]:
# 抓新聞列表
items = page.find('ul', {'id': re.compile("jsMainList\w*")}).findAll('li')
items

[<li><a href="/news/aipl/202502270050.aspx"><div class="wrap"><img alt="國民黨民調：56.7%支持還勞工7天國定假日" class="lazyload" data-src="https://imgcdn.cna.com.tw/www/WebPhotos/200/20250227/1152x768_wmky_0_C20250227000015.jpg"/></div><div class="listInfo"><h2><span>國民黨民調：56.7%支持還勞工7天國定假日</span></h2><div class="date">2025/02/27 10:45</div></div></a></li>,
 <li><a href="/news/aipl/202502270046.aspx"><div class="wrap"><img alt="賴清德指示莊瑞雄經營台東 陳瑩：他是雄讚桶箍" class="lazyload" data-src="https://imgcdn.cna.com.tw/www/WebPhotos/200/20250227/1200x675_131850584896.jpg"/></div><div class="listInfo"><h2><span>賴清德指示莊瑞雄經營台東 陳瑩：他是雄讚桶箍</span></h2><div class="date">2025/02/27 10:34</div></div></a></li>,
 <li><a href="/news/aipl/202502270044.aspx"><div class="listInfo"><h2><span>營養午餐熱量超標、沒專法規範  監察院要教育部檢討</span></h2><div class="date">2025/02/27 10:26</div></div></a></li>,
 <li><a href="/news/aipl/202502270043.aspx"><div class="wrap"><img alt="今年首個歐洲議會團訪台 團長：加強合作守護民主" class="lazyload" data-src="https://imgcdn.cna.com.tw/www/

In [ ]:
category = '政治'


# 存放資料之變數
article_link_list =[]
title_list=[]
date_list=[]
category_list=[]
photo_link_list=[]

item_id_list=[]
content_list=[]


serial_no = 1

for item in items:
    title = item.find('h2').text
    print( title )
    
    #新聞連結
    article_url = item.find('a').get('href')
    print(article_url)

    # 最新的新聞列表之新聞連結:開頭缺少"https://www.cna.com.tw"
    if article_url.startswith("/news"):
        article_url = "https://www.cna.com.tw"+article_url
        article_link_list.append(article_url)
    else:
        article_link_list.append(article_url)
    
    news_time = item.find('div', {'class':"date"}).text
    #print(dt)
    dtime = datetime.strptime(news_time, '%Y/%m/%d %H:%M')
    news_date = dtime.strftime("%Y-%m-%d")


    try:
        if item.find('img').has_attr('data-src'):
            photo_link = item.find('img').get('data-src')
        else:
            photo_link = item.find('img').get('src')
        print(photo_link)
    except:
        photo_link = ''
    
    category_list.append(category)
    date_list.append(news_date)
    title_list.append(title)
    photo_link_list.append(photo_link)
    
    # item id
    date_str = dtime.strftime("%Y%m%d")

    item_id_list.append("cna_" + date_str + "_" + str(serial_no))
    serial_no += 1

    ##### Crawl article content爬新聞內容
    print(article_url)
    # There are some words we don't like to analyze, They should be removed.
    req = requests.get(article_url, headers={ 'user-agent': user_agent.random }, timeout=5)
    
    page = BeautifulSoup(req.text,'lxml')

    # Find content
    content = page.find('div',{'class':"paragraph"})


    # Remove unnecessary content
    # Remove "透過 Google News追蹤中央社"
    try:
        content.find('div', {'class':"gmailNews"}).decompose()
    except:
        pass

    # Remove "訂閱早安電子報"
    try:
        content.find('div', {'class':"subscriInnTitle"}).decompose()
    except:
        pass

    # 額外插入的訂閱連結-->移除之
    try:
        content.find('div', {'class':"SubscriptionInner mySubscriptionInner"}).decompose()
    except:
        pass

    # 額外插入的延伸閱讀區塊-->移除之
    try:
        content.find('div', {'class':"paragraph moreArticle"}).decompose()
    except:
        pass
    # 額外插入的圖片與文字說明區塊-->移除之
    try:
        for media in content.findAll('div', {'class':"media"}):
            media.decompose()
    except:
        pass

    # Remove unnecessary content
    cont = content.text
    cont = re.sub('（中央社.+?）','', cont)
    cont = re.sub('（編輯：.*','', cont) 
    cont = re.sub('（譯者：.*','', cont) 
    
    content_list.append(cont)
    
    # crawl only 5 articles
    if serial_no > 6:
        break
    
    time.sleep(10)  # 遵守爬蟲禮節，請小睡一下

國民黨民調：56.7%支持還勞工7天國定假日
/news/aipl/202502270050.aspx
https://imgcdn.cna.com.tw/www/WebPhotos/200/20250227/1152x768_wmky_0_C20250227000015.jpg
https://www.cna.com.tw/news/aipl/202502270050.aspx
賴清德指示莊瑞雄經營台東 陳瑩：他是雄讚桶箍
/news/aipl/202502270046.aspx
https://imgcdn.cna.com.tw/www/WebPhotos/200/20250227/1200x675_131850584896.jpg
https://www.cna.com.tw/news/aipl/202502270046.aspx
營養午餐熱量超標、沒專法規範  監察院要教育部檢討
/news/aipl/202502270044.aspx
https://www.cna.com.tw/news/aipl/202502270044.aspx
今年首個歐洲議會團訪台 團長：加強合作守護民主
/news/aipl/202502270043.aspx
https://imgcdn.cna.com.tw/www/WebPhotos/200/20250227/1153x768_wmkn_0_C20250227000011.jpg
https://www.cna.com.tw/news/aipl/202502270043.aspx


In [78]:
len(article_link_list)

4

In [79]:
item_id_list

['cna_20250227_1', 'cna_20250227_2', 'cna_20250227_3', 'cna_20250227_4']

In [80]:
title_list

['國民黨民調：56.7%支持還勞工7天國定假日',
 '賴清德指示莊瑞雄經營台東 陳瑩：他是雄讚桶箍',
 '營養午餐熱量超標、沒專法規範  監察院要教育部檢討',
 '今年首個歐洲議會團訪台 團長：加強合作守護民主']

## Put them into pandas dataframe and save

#### 先將資料組合在一起，zip縫起來

In [82]:
import pandas as pd

In [83]:
data = zip(item_id_list,date_list, title_list,category_list,content_list,article_link_list,photo_link_list)
df = pd.DataFrame(list(data), columns=['item_id','date','title','category','content','link','photo_link'])

In [84]:
df.head(3)

item_id        date                      title category  \
0  cna_20250227_1  2025-02-27     國民黨民調：56.7%支持還勞工7天國定假日       政治   
1  cna_20250227_2  2025-02-27     賴清德指示莊瑞雄經營台東 陳瑩：他是雄讚桶箍       政治   
2  cna_20250227_3  2025-02-27  營養午餐熱量超標、沒專法規範  監察院要教育部檢討       政治   

                                             content  \
0  國民黨智庫今天公布民調，56.7%民眾支持還勞工7天國定假日，66.3%支持勞動節全民放假，...   
1  兼任民進黨主席的總統賴清德昨日在中常會指示立委莊瑞雄全力經營台東；積極朝向參選下屆縣長布局的...   
2  監察院今天說，教育部未落實把關學校午餐餐點，且迄今仍未訂定學校午餐專法，各級學校仍有供餐熱量...   

                                                link  \
0  https://www.cna.com.tw/news/aipl/202502270050....   
1  https://www.cna.com.tw/news/aipl/202502270046....   
2  https://www.cna.com.tw/news/aipl/202502270044....   

                                          photo_link  
0  https://imgcdn.cna.com.tw/www/WebPhotos/200/20...  
1  https://imgcdn.cna.com.tw/www/WebPhotos/200/20...  
2

In [85]:
df.shape

(4, 7)

## Save

In [ ]:
df.to_csv("cna_news_political_categroy.csv", sep="|", index=False)

## Read

In [87]:
import pandas as pd

In [88]:
df = pd.read_csv("cna_news_political_categroy.csv", sep="|")

In [89]:
df.head(2)

item_id        date                   title category  \
0  cna_20250227_1  2025-02-27  國民黨民調：56.7%支持還勞工7天國定假日       政治   
1  cna_20250227_2  2025-02-27  賴清德指示莊瑞雄經營台東 陳瑩：他是雄讚桶箍       政治   

                                             content  \
0  國民黨智庫今天公布民調，56.7%民眾支持還勞工7天國定假日，66.3%支持勞動節全民放假，...   
1  兼任民進黨主席的總統賴清德昨日在中常會指示立委莊瑞雄全力經營台東；積極朝向參選下屆縣長布局的...   

                                                link  \
0  https://www.cna.com.tw/news/aipl/202502270050....   
1  https://www.cna.com.tw/news/aipl/202502270046....   

                                          photo_link  
0  https://imgcdn.cna.com.tw/www/WebPhotos/200/20...  
1  https://imgcdn.cna.com.tw/www/WebPhotos/200/20...

In [90]:
df.content[0]

'國民黨智庫今天公布民調，56.7%民眾支持還勞工7天國定假日，66.3%支持勞動節全民放假，64.3%民眾支持教師節列國定假日。國民黨主席朱立倫表示，勞工一直在講「還假於民」，立法院要面對工時及假日問題。國民黨立法院黨團將還勞工7天國定假日列為立法院新會期的優先法案；有關國定假日天數調整議題，行政院長卓榮泰日前表示，希望能採全國一致性原則，也希望以社會付出成本最低、變化最少的方式來增加，大家能公開理性討論。國民黨智庫今天召開「恢復國定假日民調發布」記者會，智庫執行長柯志恩指出，針對還勞工7天假，民調顯示有56.7%的民眾支持，僅有19.3%民眾不支持。另外有66.3%民眾支持勞動節全民放假、64.3%民眾支持教師節列國定假日、64.1%民眾支持台灣光復節列國定假日、65.4%民眾支持行憲紀念日列國定假日。國民黨主席朱立倫表示，台灣真的是血汗之島，勞工工時在亞洲排名第2，僅次於新加坡，但新加坡的薪水是台灣的好幾倍。觀察鄰近國家，日本國定假日有16天，韓國15天，香港17天，台灣只有12天。朱立倫指出，台灣的國定假日變少，跟當年一例一休修法有很大的關係，也讓台灣勞工一直在講「還假於民，還勞工7天假」。至於應該怎麼做，立法院要好好面對勞工工時及國定假日的問題。與會的國民黨立法院黨團首席副書記長羅智強表示，希望賴政府能夠正面、大方且柔軟的回應社會大眾對於還假的議題，不要讓勞工的肝變成最硬的那一塊，拜託民進黨政府傾聽民意，還假於民。國民黨立委許宇甄指出，過去蔡政府取消勞工7天假，引發勞工不滿，代表民進黨政府制定勞工政策沒有考慮勞工的相關權益。台灣工時過長是非常嚴重的問題，要解決工時過長及特休休不到的問題，最快速的方法是立法增加國定假日。國民黨這份民調委託十分民調公司執行市話民調，執行時間為2月20日至22日，有效樣本為1068份，抽樣誤差在95%信心水準下，為正負3%。'